In [1]:
import pandas as pd
import os
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MeanShift
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification

from joblib import dump, load

In [2]:
base = pd.read_csv('resultats_google_lemmatises_2005_complet.csv')

In [3]:
base.head()

,Unnamed: 0,Date_debut,Date_fin,Pays,Rang,Liste_URL,Contenu_page,Mots_retenus,Mots_lemm
0,0,2010-01-01,2010-12-31,Afrique du Sud,0,http://www.slate.fr/story/26143/afrique-du-sud...,googletag cmd push function googletag disp...,"['issue', 'peche', 'chair', 'partage', 'eclips...",issue peche chair partage eclipse uni verre ta...
1,1,2010-01-01,2010-12-31,Afrique du Sud,1,https://www.geo.fr/voyage/voyage-en-afrique-du...,sprite symbol usage display none spri...,"['salade', 'peche', 'recherchees', 'ancien', '...",salade peche recherchees ancien nombre photo p...
2,2,2010-01-01,2010-12-31,Afrique du Sud,2,http://plus.lefigaro.fr/tag/afrique-du-sud,fig header figh full fig header collapsed ...,"['trimestre', 'archeveque', 'veteran', 'lutte'...",trimestre archeveque veteran lutte moi site ch...
3,3,2010-01-01,2010-12-31,Afrique du Sud,3,https://www.quandpartir.com/meteo/afrique-du-s...,abonnement aux bons plans voyage ...,"['remontant', 'devises', 'tableau', 'antilopes...",remontant devise tableau antilope active catch...
4,4,2010-01-01,2010-12-31,Afrique du Sud,4,https://www.franceculture.fr/histoire/lafrique...,programmespodcastsactualitessavoirsart et crea...,"['part', 'derivee', 'roche', 'rush', 'bain', '...",part derivee roche rush bain dura etoile piece...


In [3]:
def get_merged_csv(f_list, **kwargs):
    return pd.concat([pd.read_csv(f, **kwargs) for f in f_list], ignore_index=True)

base = get_merged_csv(glob.glob('base_lemmatisee*.csv'), index_col=None)
base.head()

C:\Users\lleki\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Contenu_page,Date_debut,Date_fin,Index,Liste_URL,Mots_lemm,Mots_retenus,Pays,Rang,Serie_mots,Unnamed: 0,Unnamed: 0.1
0,retour a la page d accueil qui somme...,2005-01-01,2005-12-31,0.0,https://www.clio.fr/voyages_culturels/bibliogr...,jour recits actualites geopolitique decouverte...,"['jours', 'recits', 'actualites', 'geopolitiqu...",Ouzbékistan,0,NaN,0,0.0
1,please enable js and disable any ad blockervar...,2005-01-01,2005-12-31,1.0,https://www.cairn.info/revue-le-courrier-des-p...,p o u b e l l e,poubelle,Ouzbékistan,1,NaN,1,1.0
2,destinations ...,2005-01-01,2005-12-31,2.0,https://www.evaneos.fr/ouzbekistan/voyage/etap...,historique sang jour centre demande longitude ...,"['historique', 'sang', 'jours', 'centre', 'dem...",Ouzbékistan,2,NaN,2,2.0
3,forbidden you don t have permission to access ...,2005-01-01,2005-12-31,3.0,https://www.lechotouristique.com/article/sur-l...,p o u b e l l e,poubelle,Ouzbékistan,3,NaN,3,3.0
4,englishespanolfrancais un site de ressources...,2005-01-01,2005-12-31,4.0,http://www.irenees.net/bdf_fiche-experience-19...,heritage separees espace couteux oublier predi...,"['heritage', 'separees', 'espace', 'couteux', ...",Ouzbékistan,4,NaN,4,4.0


In [4]:
base.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [5]:
base.head()

,Date_debut,Date_fin,Pays,Rang,Liste_URL,Contenu_page,Mots_retenus,Mots_lemm
0,2010-01-01,2010-12-31,Afrique du Sud,0,http://www.slate.fr/story/26143/afrique-du-sud...,googletag cmd push function googletag disp...,"['issue', 'peche', 'chair', 'partage', 'eclips...",issue peche chair partage eclipse uni verre ta...
1,2010-01-01,2010-12-31,Afrique du Sud,1,https://www.geo.fr/voyage/voyage-en-afrique-du...,sprite symbol usage display none spri...,"['salade', 'peche', 'recherchees', 'ancien', '...",salade peche recherchees ancien nombre photo p...
2,2010-01-01,2010-12-31,Afrique du Sud,2,http://plus.lefigaro.fr/tag/afrique-du-sud,fig header figh full fig header collapsed ...,"['trimestre', 'archeveque', 'veteran', 'lutte'...",trimestre archeveque veteran lutte moi site ch...
3,2010-01-01,2010-12-31,Afrique du Sud,3,https://www.quandpartir.com/meteo/afrique-du-s...,abonnement aux bons plans voyage ...,"['remontant', 'devises', 'tableau', 'antilopes...",remontant devise tableau antilope active catch...
4,2010-01-01,2010-12-31,Afrique du Sud,4,https://www.franceculture.fr/histoire/lafrique...,programmespodcastsactualitessavoirsart et crea...,"['part', 'derivee', 'roche', 'rush', 'bain', '...",part derivee roche rush bain dura etoile piece...


In [6]:
base_clean = base[base.Mots_retenus != "poubelle"]

In [11]:
base_clean[base.Date_debut == '2019-01-01'].head()

C:\Users\lleki\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Contenu_page,Date_debut,Date_fin,Liste_URL,Mots_lemm,Mots_retenus,Pays,Rang
14593,le point montres le point pop auto vin phebe s...,2019-01-01,2019-12-31,https://www.lepoint.fr/afrique/afrique-du-sud-...,niveau issue integralite inclus part legislati...,"['niveaux', 'issue', 'integralite', 'inclus', ...",Afrique du Sud,0
14594,consulterle journal navigation le mon...,2019-01-01,2019-12-31,https://www.lemonde.fr/afrique/article/2019/10...,circulation partage chanson decodeurs uni abat...,"['circulation', 'partage', 'chansons', 'decode...",Afrique du Sud,1
14595,consulterle journal navigation le mon...,2019-01-01,2019-12-31,https://www.lemonde.fr/afrique/article/2019/09...,surprise circulation partage chanson decodeurs...,"['surprise', 'circulation', 'partage', 'chanso...",Afrique du Sud,2
14596,font face font display swap font family source...,2019-01-01,2019-12-31,https://www.lefigaro.fr/flash-actu/l-afrique-d...,uni regles sert pieces meta active livre partn...,"['unis', 'regles', 'sert', 'pieces', 'meta', '...",Afrique du Sud,3
14597,franceinfoafriqueaccueila la unepolitique afri...,2019-01-01,2019-12-31,https://www.francetvinfo.fr/monde/afrique/afri...,district pris chauffeur immigrant provoquant m...,"['district', 'pris', 'chauffeurs', 'immigrants...",Afrique du Sud,4


In [7]:
base_clean.to_csv('resultats_google_lemmatises_2005_complet.csv')

# Modèle NMF

In [8]:
stopwords = ['site','document','window','menu','recherche','mentions','script','accueil']

for nb_clusters in range(4,21):
    modelNMF = NMF(n_components=nb_clusters, init='random', random_state=0)

    pipelineNMF = Pipeline(
      steps=[
        ('tfidf', TfidfVectorizer(stop_words = stopwords)),
        ('clust', modelNMF)])

    pipelineNMF.fit(base_clean.Mots_lemm)
    dump(pipelineNMF, 'NMF'+str(nb_clusters)+'.joblib')

# Modèle LDA

In [9]:
for nb_clusters in range(4,21):
        modelLDA = LatentDirichletAllocation(n_components=nb_clusters, random_state=0, learning_method = 'batch')
        pipelineLDA = Pipeline(
          steps=[
            ('tfidf', TfidfVectorizer(stop_words = stopwords)),
            ('clust', modelLDA)])

        pipelineLDA.fit(base_clean.Mots_lemm)
        dump(pipelineLDA, 'LDA'+str(nb_clusters)+'.joblib')